In [325]:
from math import factorial, floor, ceil

def print_table(arr, rows=[], columns=[]):
    if len(rows) > 0:
        for i in range(len(rows)):
            arr[i] = [str(rows[i])] + arr[i]

    if len(columns) > 0:
        arr = [[' '] + list(map(str, columns))] + arr

    row_lengths = [max(map(lambda x: len(str(x)), column)) for column in zip(*arr)]

    for row in arr:
        for i in range(len(row)):
            padding = row_lengths[i] - len(str(row[i])) + 2
            print(f'{" " * floor(padding / 2)}{row[i]}{" " * ceil(padding / 2)}', end='')
        print()

def calculate(lamb, mu, n, assistance=False):
    ro = lamb / mu
    p0 = 1 / sum((1 / factorial(k)) * ro ** k for k in range(n))

    # p_cancelation = (1 / factorial(n) * (lamb / (n * mu))**n) * (p0 ** n if assistance else p0)
    p_cancelation = (ro ** n) / factorial(n) * (p0 ** n if assistance else p0)
    # p_cancelation = 1 / (1 + sum([(ro ** k) / factorial(k) for k in range(n)]))
    Q = 1 - p_cancelation
    A = lamb * Q
    n_o = A / mu

    return [p_cancelation, Q, A, n_o]
    
n = 3
lamb = 4
mu = 2

no_assistance = calculate(lamb, mu, n)
unlimited_assistance = calculate(lamb, mu, n, assistance=True)
uniform_assistance = calculate(lamb, mu, n, assistance=True)

print_table([no_assistance, unlimited_assistance, uniform_assistance], 
            rows=['Без взаимопомощи', 'Неограниченная', 'Равномерная'], 
            columns=['Вероятность отказа', 'Относительная пропускная способность', 'Абсолютая пропускная способность', 'Среднее число занятых каналов'])

                    Вероятность отказа   Относительная пропускная способность  Абсолютая пропускная способность  Среднее число занятых каналов 
 Без взаимопомощи  0.26666666666666666            0.7333333333333334                  2.9333333333333336              1.4666666666666668       
  Неограниченная   0.010666666666666668           0.9893333333333333                  3.957333333333333               1.9786666666666666       
   Равномерная     0.010666666666666668           0.9893333333333333                  3.957333333333333               1.9786666666666666       


In [327]:
import simpy
import random

def source(env, arrival_rate, service_rate, channels):
    global total_requests, rejected_requests
    total_requests = 0
    rejected_requests = 0
    
    while True:
        yield env.timeout(random.expovariate(arrival_rate))
        total_requests += 1
        env.process(customer(env, channels, service_rate))

def customer(env, channels, service_rate):
    global rejected_requests
    with channels.request() as req:
        results = yield req | env.timeout(0, value='rejected')
        
        if req not in results:
            rejected_requests += 1
            return

        yield env.timeout(random.expovariate(service_rate))

def simulate(arrival_rate, service_rate, channels_count, sim_time):
    env = simpy.Environment()
    channels = simpy.Resource(env, capacity=channels_count)
    env.process(source(env, arrival_rate, service_rate, channels))
    env.run(until=sim_time)

    probability_of_rejection = rejected_requests / total_requests if total_requests > 0 else 0

    print("Total requests:", total_requests)
    print("Rejected requests:", rejected_requests)
    print("Probability of rejection:", probability_of_rejection)

# Параметры симуляции
X = 4  # Заявки в минуту
Y = 0.5  # Среднее время обслуживания одним каналом (в минутах)
channels_count = 3  # Количество каналов
sim_time = 10000  # Время симуляции (в минутах)

# Интенсивность обслуживания заявки к каналами
service_rate = channels_count / Y

# Запуск симуляции
simulate(X, service_rate, channels_count, sim_time)


Total requests: 39896
Rejected requests: 1005
Probability of rejection: 0.025190495287748145
